In [1]:
import sqlalchemy as sqla
import getpass
import pymssql

user = 'tsquare_app_user'
password = getpass.getpass()
server = 'tsquare-public.cwgloob09acb.us-east-1.rds.amazonaws.com'
database = 'TSQUARE'
engine = sqla.create_engine('mssql+pymssql://' + user + ':' + password + '@' + server + '/' + database)
connection = engine.connect()
print(connection)

········


In [2]:
import pandas as pd

query = """
SELECT
	FlowStateMatchup.pf_enc,
	PatientFlowFeatures.LocationCode,
	PatientFlowFeatures.DoctorNameLast,
	PatientFlowFeatures.VisitType,
	FlowStateMatchup.pf_currentstate,
	FlowStateMatchup.pf_nextstate,
	FlowStateMatchup.pf_stateduration
FROM
(
SELECT
	CurrentFlowState.pfid AS pfid,
	CurrentFlowState.pf_enc,
	CurrentFlowState.pf_state AS pf_currentstate,
	CASE CurrentFlowState.pf_state
		WHEN 'Check-out'
		THEN 'Check-out'
		ELSE COALESCE(NextFlowState.pf_state, 'Check-out') 
		END
	AS pf_nextstate,
	CurrentFlowState.pf_time,
	DATEDIFF(mi, CurrentFlowState.pf_time, COALESCE(NextFlowState.pf_time, CurrentFlowState.pf_time)) AS pf_stateduration,
	ROW_NUMBER() OVER (
				PARTITION BY CurrentFlowState.pf_enc,
				CASE CurrentFlowState.pf_state
					WHEN 'Check-out'
					THEN 'Check-out'
					ELSE COALESCE(NextFlowState.pf_state, 'Check-out') 
					END,
				CurrentFlowState.pf_time
				ORDER BY CurrentFlowState.pf_time) 
	AS uniqueness 
FROM
(
SELECT
	flow.Id AS pfid,
	flow.EncounterNumber AS pf_enc,
	pftype.[Name] AS pf_state,
	flow.EventTime AS pf_time,
	DENSE_RANK() OVER (PARTITION BY flow.EncounterNumber ORDER BY flow.EventTime ASC) AS pf_statehierarchy
FROM
	Facts.EncounterPatientFlow flow
INNER JOIN
	Dimensions.PatientFlowType pftype
ON
	flow.PatientFlowType_Id = pftype.Id
) AS CurrentFlowState
LEFT JOIN
(
SELECT
	flow.Id AS pfid,
	flow.EncounterNumber AS pf_enc,
	pftype.[Name] AS pf_state,
	flow.EventTime AS pf_time,
	DENSE_RANK() OVER (PARTITION BY flow.EncounterNumber ORDER BY flow.EventTime ASC) - 1 AS pf_statehierarchy
FROM
	Facts.EncounterPatientFlow flow
INNER JOIN
	Dimensions.PatientFlowType pftype
ON
	flow.PatientFlowType_Id = pftype.Id
) AS NextFlowState
ON
	CurrentFlowState.pf_enc = NextFlowState.pf_enc
	AND CurrentFlowState.pf_statehierarchy = NextFlowState.pf_statehierarchy
WHERE 
	YEAR(CurrentFlowState.pf_time) >= 2017
AND
	(
	CurrentFlowState.pf_state = 'Check-out' 
	OR 
	CurrentFlowState.pf_state != NextFlowState.pf_state
	)
GROUP BY
	CurrentFlowState.pfid,
	CurrentFlowState.pf_enc,
	CurrentFlowState.pf_state,
	CASE CurrentFlowState.pf_state
		WHEN 'Check-out'
		THEN 'Check-out'
		ELSE COALESCE(NextFlowState.pf_state, 'Check-out') 
		END,
	CurrentFlowState.pf_time,
	DATEDIFF(mi, CurrentFlowState.pf_time, COALESCE(NextFlowState.pf_time, CurrentFlowState.pf_time))
) FlowStateMatchup
INNER JOIN
	(
	SELECT
		pf.Id AS pfid,
		doc.DoctorNameLast,
		vis.[Name] AS VisitType,
		loc.LocationCode
	FROM
		Facts.EncounterPatientFlow pf
	INNER JOIN
		Dimensions.Doctor doc
	ON
		pf.Doctor_Id = doc.Id
	INNER JOIN
		Dimensions.VisitType vis
	ON
		pf.VisitType_Id = vis.Id
	INNER JOIN
		Dimensions.LocationPod loc
	ON
		pf.LocationPod_Id = loc.Id
	GROUP BY
		pf.Id,
		doc.DoctorNameLast,
		vis.[Name],
		loc.LocationCode
	) PatientFlowFeatures
ON
	FlowStateMatchup.pfid = PatientFlowFeatures.pfid
WHERE 
	uniqueness=1
"""
    
df_master = pd.read_sql_query(sql = query , con=connection)
df_master.head()

,pf_enc,LocationCode,DoctorNameLast,VisitType,pf_currentstate,pf_nextstate,pf_stateduration
0,981724,Huntersville,Kramer,Adult Strabismus,Wait,Tech,42
1,1039442,Cotswold,Krug Jr,CE Routine,Check-out,Check-out,12
2,977753,Cotswold,James,CE Routine,Check-in,Wait,3
3,1139979,Pineville,Weidman,Retina Follow Up,Wait,Special Testing,7
4,1081018,Cotswold,Syracuse,Follow Up/Office Visit,Provider,Check-out,4


In [69]:
from collections import defaultdict

predict_vars = ['LocationCode', 'DoctorNameLast', 'VisitType']
other_columns = list(df_master.columns)
for var in predict_vars:
    other_columns.remove(var)

def bootstrap_sampler(source, split_col='',replace=True, frac=0.4):
    sample = defaultdict(object)
    temp_list = list(other_columns)
    temp_list.append(split_col)
    df_slice = df_master.filter(items=temp_list, axis=1)
    for value in df_slice[split_col].unique():
        sample[value] = df_slice[df_slice[split_col] == value].sample(frac=frac, replace=replace)
    return sample

In [70]:
big_ol_boots = defaultdict(list)

for feature in predict_vars:
    big_ol_boots[feature] = bootstrap_sampler(df_master, split_col=feature)

In [71]:
doctors = df_master.DoctorNameLast.unique()
locations = df_master.LocationCode.unique()
visit_types = df_master.VisitType.unique()

In [93]:
def calculate_average_state_duration(source,pred_var=''):
    df_times = source.filter(items=[pred_var,'pf_currentstate', 'pf_stateduration'])
    df_times_reset = df_times.groupby([pred_var, 'pf_currentstate']).mean().reset_index(col_fill=['avg_state_duration'])
    return df_times_reset

In [94]:
calculate_average_state_duration(big_ol_boots['DoctorNameLast']['Kramer'], pred_var='DoctorNameLast')

,DoctorNameLast,pf_currentstate,pf_stateduration
0,Kramer,Arrived,6.481226
1,Kramer,Check-in,4.108571
2,Kramer,Check-out,5.075287
3,Kramer,Contact Lens,27.600000
4,Kramer,Dilating,36.648879
5,Kramer,Other,73.263158
6,Kramer,Provider,12.764970
7,Kramer,Special Testing,16.459459
8,Kramer,Surgery Scheduling,24.200000
9,Kramer,Tech,15.604465


In [129]:
def calculate_transition_probabilities(source, pred_var=''):
    df_transitions = source.filter(items=[pred_var, 'pf_currentstate', 'pf_nextstate'])
    df_transitions = df_transitions.groupby([pred_var, 'pf_currentstate', 'pf_nextstate']).size().reset_index()
    new_names = list(df_transitions.columns) 
    new_names[-1] = 'TransitionCount'
    df_transitions.columns = new_names
    df_instances = source.filter(items=[pred_var, 'pf_currentstate'])
    df_instances = df_instances.groupby([pred_var, 'pf_currentstate']).size().reset_index()
    new_names = list(df_instances.columns)
    new_names[-1] = 'InstanceCount'
    df_instances.columns = new_names
    df_probabilities = df_transitions.merge(df_instances,
                                            left_on='pf_currentstate',
                                            right_on='pf_currentstate')
    df_probabilities['TransitionProbability'] = df_probabilities.TransitionCount/df_probabilities.InstanceCount
    return df_probabilities

In [152]:
probs = calculate_transition_probabilities(big_ol_boots['DoctorNameLast']['Kramer'], pred_var='DoctorNameLast')

In [176]:
states = list(df_master['pf_currentstate'].unique())
states.sort()
states.append(states.pop(2))
states

['Arrived',
 'Check-in',
 'Contact Lens',
 'Dilating',
 'Other',
 'Provider',
 'Special Testing',
 'Surgery Scheduling',
 'Tech',
 'Wait',
 'Check-out']

In [166]:
probs

,DoctorNameLast_x,pf_currentstate,pf_nextstate,TransitionCount,DoctorNameLast_y,InstanceCount,TransitionProbability
0,Kramer,Arrived,Check-in,1244,Kramer,1305,0.953257
1,Kramer,Arrived,Check-out,1,Kramer,1305,0.000766
2,Kramer,Arrived,Tech,6,Kramer,1305,0.004598
3,Kramer,Arrived,Wait,54,Kramer,1305,0.041379
4,Kramer,Check-in,Arrived,4,Kramer,1750,0.002286
5,Kramer,Check-in,Check-out,5,Kramer,1750,0.002857
6,Kramer,Check-in,Dilating,5,Kramer,1750,0.002857
7,Kramer,Check-in,Other,4,Kramer,1750,0.002286
8,Kramer,Check-in,Special Testing,3,Kramer,1750,0.001714
9,Kramer,Check-in,Tech,127,Kramer,1750,0.072571


In [186]:
import numpy as np

P = np.zeros((len(states),len(states)))

In [187]:
for i in range(len(states)):
    for j in range(len(states)):
        try: 
            P[i][j] = probs[(probs.pf_currentstate == states[i]) & (probs.pf_nextstate == states[j])].reset_index().get_value(index=0,col='TransitionProbability')
        except KeyError:
            P[i][j] = 0
            
fund_mtrx

array([[  0.00000000e+00,   9.53256705e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   4.59770115e-03,
          4.13793103e-02,   7.66283525e-04],
       [  2.28571429e-03,   0.00000000e+00,   0.00000000e+00,
          2.85714286e-03,   2.28571429e-03,   0.00000000e+00,
          1.71428571e-03,   0.00000000e+00,   7.25714286e-02,
          9.15428571e-01,   2.85714286e-03],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   7.73395205e-04,
          0.00000000e+00,   0.00000000e+00,   1.73240526e-01,
          3.09358082e-03,   0.00000000e+00,   0.00000000e+00,
          7.31631864e-01,   9.12606342e-02],
       [  5.26315789e-02,   0.00000000e+00,   0.00000000e+00,
          0.00

In [188]:
np.sum(P, axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [2]:
import numpy as np
np.ones(11)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [189]:
Q = P[:-1,:-1]

In [191]:
fund_mtx = np.linalg.inv(np.identity(10) - Q)

In [192]:
fund_mtx

array([[  1.00850469e+00,   1.00943401e+00,   2.61866009e-03,
          8.90146855e-01,   9.08640140e-03,   9.32069541e-01,
          2.15307311e-02,   1.31394081e-03,   1.09390613e+00,
          2.15083105e+00],
       [  8.62129822e-03,   1.05632507e+00,   2.62020781e-03,
          8.90349484e-01,   9.19874451e-03,   9.32673662e-01,
          2.16243306e-02,   1.31486241e-03,   1.09354884e+00,
          2.15246930e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  5.06992848e-03,   4.23647337e-02,   2.95299363e-03,
          1.70308585e+00,   5.38977847e-03,   8.99771428e-01,
          1.92945566e-02,   1.15465841e-03,   7.95580731e-01,
          1.67930703e+00],
       [  5.81611845e-02,   9.16655895e-02,   2.43905761e-03,
          9.95575891e-01,   1.00531790e+00,   9.89024531e-01,
          1.71552324e-02

In [193]:
df_times = calculate_average_state_duration(big_ol_boots['DoctorNameLast']['Kramer'], pred_var='DoctorNameLast')

In [195]:
df_times2 = df_times[df_times.pf_currentstate != 'Check-out']
times_matrix = np.array(df_times2.sort_values(by='pf_currentstate').pf_stateduration)
times_matrix

array([  6.48122605,   4.10857143,  27.6       ,  36.64887858,
        73.26315789,  12.76497006,  16.45945946,  24.2       ,
        15.6044648 ,  15.20859599])

In [203]:
fund_mtx[0].dot(times_matrix.T)

106.10949768186555

In [225]:
df_visits = df_master.filter(['DoctorNameLast', 'pf_enc', 'pf_stateduration'], axis=1)
df_visits = df_visits.groupby(['DoctorNameLast', 'pf_enc']).sum().reset_index()
df_visits = df_visits[(df_visits['DoctorNameLast'] == 'Kramer') & (df_visits.pf_stateduration < 500)].sample(replace=True, frac=.2)
df_visits['sq_deviations'] = (df_visits["pf_stateduration"] - 106.11)**2

In [226]:
df_visits

,DoctorNameLast,pf_enc,pf_stateduration,sq_deviations
33749,Kramer,1121242,105,1.2321
34455,Kramer,1145386,190,7037.5321
33194,Kramer,1099807,130,570.7321
32139,Kramer,1053503,65,1690.0321
33646,Kramer,1117568,104,4.4521
33727,Kramer,1120029,67,1529.5921
31030,Kramer,1002084,74,1031.0521
31648,Kramer,1031679,115,79.0321
31514,Kramer,1026965,85,445.6321
31448,Kramer,1021758,56,2511.0121


In [228]:
sq_dev = np.array(df_visits['sq_deviations'])

In [237]:
np.sqrt(sq_dev.sum()/911)/106

0.40507550464467207

911